<a href="https://colab.research.google.com/github/sarahxu087/DS-Unit-2-Linear-Models/blob/master/module4-logistic-regression/Sarah_Xu__assignment_regression_classification_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [ ] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [ ] Begin with baselines for classification.
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [ ] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [0]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [0]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [0]:
# Drop some high cardinality categoricals
df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [0]:
# Drop some columns to prevent "leakage"
df = df.drop(columns=['Rec', 'overall'])

In [0]:
df

,Burrito,Date,Yelp,Google,Chips,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Unreliable,NonSD,Beef,Pico,Guac,Cheese,Fries,Sour cream,Pork,Chicken,Shrimp,Fish,Rice,Beans,Lettuce,Tomato,Bell peper,Carrots,Cabbage,Sauce,Salsa.1,Cilantro,Onion,Taquito,Pineapple,Ham,Chile relleno,Nopales,Lobster,Queso,Egg,Mushroom,Bacon,Sushi,Avocado,Corn,Zucchini,Great
0,California,1/18/2016,3.5,4.2,NaN,6.49,3.0,NaN,NaN,NaN,NaN,NaN,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1,California,1/24/2016,3.5,3.3,NaN,5.45,3.5,NaN,NaN,NaN,NaN,NaN,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0,NaN,NaN,x,x,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,Carnitas,1/24/2016,NaN,NaN,NaN,4.85,1.5,NaN,NaN,NaN,NaN,NaN,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0,NaN,NaN,NaN,x,x,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,Asada,1/24/2016,NaN,NaN,NaN,5.25,2.0,NaN,NaN,NaN,NaN,NaN,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0,NaN,NaN,x,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,California,1/27/2016,4.0,3.8,x,6.59,4.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0,NaN,NaN,x,x,NaN,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,Other,8/27/2019,NaN,NaN,NaN,6.00,1.0,NaN,NaN,17.0,20.5,0.57,5.0,4.0,3.5,NaN,4.0,4.0,2.0,2.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
419,Other,8/27/2019,NaN,NaN,NaN,6.00,4.0,NaN,NaN,19.0,26.0,1.02,4.0,5.0,NaN,3.5,4.0,4.0,5.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
420,California,8/27/2019,NaN,NaN,NaN,7.90,3.0,NaN,NaN,20.0,22.0,0.77,4.0,4.0,4.0,3.7,3.0,2.0,3.5,4.0,4.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
421,Other,8/27/2019,NaN,NaN,NaN,7.90,3.0,NaN,NaN,22.5,24.5,1.07,5.0,2.0,5.0,5.0,5.0,2.0,5.0,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [0]:
df.shape

(421, 59)

In [0]:
df.isna().sum()

Burrito             0
Date                0
Yelp              334
Google            334
Chips             395
Cost                7
Hunger              3
Mass (g)          399
Density (g/mL)    399
Length            138
Circum            140
Volume            140
Tortilla            0
Temp               20
Meat               14
Fillings            3
Meat:filling        9
Uniformity          2
Salsa              25
Synergy             2
Wrap                3
Unreliable        388
NonSD             414
Beef              242
Pico              263
Guac              267
Cheese            262
Fries             294
Sour cream        329
Pork              370
Chicken           400
Shrimp            400
Fish              415
Rice              385
Beans             386
Lettuce           410
Tomato            414
Bell peper        414
Carrots           420
Cabbage           413
Sauce             383
Salsa.1           414
Cilantro          406
Onion             404
Taquito           417
Pineapple 

In [0]:
#clear Nan values
df=df.loc[:,df.isna().sum()<50]
df

df['Cost'].fillna((df['Cost'].mean()),inplace=True)
df['Hunger'].fillna((df['Hunger'].mean()),inplace=True)
df['Temp'].fillna((df['Temp'].mean()),inplace=True)
df['Meat'].fillna((df['Meat'].mean()),inplace=True)
df['Fillings'].fillna((df['Fillings'].mean()),inplace=True)
df['Meat:filling'].fillna((df['Meat:filling'].mean()),inplace=True)
df['Uniformity'].fillna((df['Uniformity'].mean()),inplace=True)
df['Salsa'].fillna((df['Salsa'].mean()),inplace=True)
df['Synergy'].fillna((df['Synergy'].mean()),inplace=True)
df['Wrap'].fillna((df['Wrap'].mean()),inplace=True)
df.isna().sum()
df

,Burrito,Date,Cost,Hunger,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap,Great
0,California,2016-01-18,6.49,3.0,3.0,5.0,3.000000,3.500000,4.0,4.0,4.0,4.0,4.0,False
1,California,2016-01-24,5.45,3.5,2.0,3.5,2.500000,2.500000,2.0,4.0,3.5,2.5,5.0,False
2,Carnitas,2016-01-24,4.85,1.5,3.0,2.0,2.500000,3.000000,4.5,4.0,3.0,3.0,5.0,False
3,Asada,2016-01-24,5.25,2.0,3.0,2.0,3.500000,3.000000,4.0,5.0,4.0,4.0,5.0,False
4,California,2016-01-27,6.59,4.0,4.0,5.0,4.000000,3.500000,4.5,5.0,2.5,4.5,4.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,Other,2019-08-27,6.00,1.0,5.0,4.0,3.500000,3.539833,4.0,4.0,2.0,2.0,5.0,False
419,Other,2019-08-27,6.00,4.0,4.0,5.0,3.620393,3.500000,4.0,4.0,5.0,4.0,3.0,True
420,California,2019-08-27,7.90,3.0,4.0,4.0,4.000000,3.700000,3.0,2.0,3.5,4.0,4.5,False
421,Other,2019-08-27,7.90,3.0,5.0,2.0,5.000000,5.000000,5.0,2.0,5.0,5.0,2.0,True


In [0]:
# find a train , validate , test 
df['Date']=pd.to_datetime(df['Date'])
train=df[df.Date.dt.year <=2016]
val=df[df.Date.dt.year==2017]
test=df[df.Date.dt.year>=2018]

In [0]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
Great,298,2,False,176,NaT,NaT
Burrito,298,5,California,118,NaT,NaT
Date,298,110,2016-08-30 00:00:00,29,2011-05-16,2016-12-15


In [97]:
val.shape

(85, 14)

In [95]:
# maiority class(train baseline)
target='Great'

y_train = train[target]
y_train.value_counts()
majority_class= y_train.mode([0])
print(majority_class)

y_pred=[majority_class]*len(y_train)

from sklearn.metrics import accuracy_score
accuracy_score(y_train,y_pred)


0    False
dtype: bool


0.5906040268456376

In [96]:
# maiority class(validation baseline)
y_val= val[target]
y_val.value_counts()
majority_class= y_val.mode([0])
print(majority_class)
y_pred=[majority_class]*len(y_val)
accuracy_score(y_val,y_pred)

0    False
dtype: bool


0.5529411764705883

In [83]:
high_cardinality=['Date']
features=train.columns.drop([target]+high_cardinality)
X_train = train[features]
X_val = val[features]
X_test=test[features]
X_train

,Burrito,Cost,Hunger,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap
0,California,6.49,3.0,3.0,5.0,3.0,3.5,4.0,4.0,4.00000,4.0,4.0
1,California,5.45,3.5,2.0,3.5,2.5,2.5,2.0,4.0,3.50000,2.5,5.0
2,Carnitas,4.85,1.5,3.0,2.0,2.5,3.0,4.5,4.0,3.00000,3.0,5.0
3,Asada,5.25,2.0,3.0,2.0,3.5,3.0,4.0,5.0,4.00000,4.0,5.0
4,California,6.59,4.0,4.0,5.0,4.0,3.5,4.5,5.0,2.50000,4.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
296,California,5.65,3.0,4.0,1.5,2.0,3.0,4.2,4.0,3.00000,2.0,4.5
297,Other,5.49,3.0,4.5,5.0,2.0,2.0,2.5,3.5,3.00000,2.5,3.0
298,California,7.75,4.0,3.5,2.5,3.0,3.3,1.4,2.3,2.20000,3.3,4.5
299,Asada,7.75,4.0,4.0,4.5,2.0,2.0,3.5,3.5,2.00000,2.0,4.0


In [0]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)
X_val= encoder.transform(X_val)
X_test= encoder.transform(X_test)

In [106]:
#logistic regression
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression(solver='lbfgs')
log_reg.fit(X_train,y_train)

log_reg.predict(X_val)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


array([False, False,  True,  True, False, False,  True,  True, False,
        True,  True, False, False, False,  True, False, False,  True,
        True,  True,  True, False, False,  True,  True,  True,  True,
       False,  True,  True,  True, False, False, False, False, False,
        True,  True, False,  True, False,  True,  True,  True,  True,
        True, False, False,  True, False, False, False,  True,  True,
       False, False, False, False, False, False, False, False, False,
       False, False,  True,  True,  True,  True, False,  True,  True,
        True,  True, False, False, False,  True,  True,  True,  True,
        True, False,  True,  True])

In [107]:
# Validation accuracy 

print('Validation Accuracy',log_reg.score(X_val,y_val))

Validation Accuracy 0.8352941176470589


In [108]:
#Test
log_reg=LogisticRegression(solver='lbfgs')
log_reg.fit(X_train,y_train)
print('Test Accuracy',log_reg.score(X_test,y_test))
log_reg.predict(X_test)

Test Accuracy 0.7894736842105263


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


array([ True,  True, False,  True, False, False,  True,  True, False,
        True,  True, False,  True, False,  True,  True,  True, False,
       False, False, False,  True,  True,  True,  True, False, False,
        True,  True,  True, False,  True,  True, False,  True,  True,
        True,  True])